From the cleaning data exercise, we know we have information about the passengers sex, age, class they travelled in as well as port of embarkation. We will choose these columns for our initial analysis. It is possible we will need to use others for refinements. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sklearn
import seaborn as sb

Repeating some of the steps done in cleaning. Loading the data, making the data frame, replacing NA values for Age and assigning numeric codes for Sex and Embarked, all have to be repeated.

In [3]:
trainset = pd.read_csv("train.csv")
trainframe = pd.DataFrame(trainset)

In [4]:
trainframe["Age"]= trainframe["Age"].fillna(trainframe["Age"].median())

In [5]:
trainframe.loc[trainframe["Sex"] =="male", "Sex"] = 0
trainframe.loc[trainframe["Sex"] == "female", "Sex"] = 1

In [6]:
trainframe.loc[trainframe["Embarked"] == "S", "Embarked"] = 0
trainframe.loc[trainframe["Embarked"] == "C", "Embarked"] = 1
trainframe.loc[trainframe["Embarked"] == "Q", "Embarked"] = 2

In [7]:
trainframe["Embarked"] = trainframe["Embarked"].fillna(trainframe["Embarked"].median())

Now that the data frame is cleaned and ready for modeling, I want to split the training set into training and cross validation sets. 

In [8]:
from sklearn.cross_validation import train_test_split

In [9]:
trainX = trainframe[["Pclass", "Sex", "SibSp", "Parch", "Fare", "Embarked" ]]
trainY = trainframe[["Survived"]]

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(trainX, trainY, test_size = 0.2, train_size = 0.8, random_state = 1)

I will have to do multiple training and cross validation sets, but for now I want to one. I will extend it to more later.

In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
model = LinearRegression()

In [13]:
model.fit(X_train, Y_train )

LinearRegression(copy_X=True, fit_intercept=True, normalize=False)

In [14]:
test_predictions = model.predict(X_test)

In [15]:
predictions = test_predictions
predictions[test_predictions < 0.5] = 0
predictions[test_predictions >=0.5] = 1

In [16]:
model

LinearRegression(copy_X=True, fit_intercept=True, normalize=False)

In [17]:
predictions = test_predictions
predictions[test_predictions < 0.2] = 0
predictions[test_predictions >=0.8] = 1

In [18]:
accurracy = sum(predictions[predictions == Y_test])/ len(predictions)

In [19]:
print(accurracy)

0.2681564245810056


In [20]:
predictions = test_predictions
predictions[test_predictions < 0.5] = 0
predictions[test_predictions >=0.5] = 1

In [21]:
accurracy = sum(predictions[predictions == Y_test])/ len(predictions)

In [22]:
print(accurracy)

0.2681564245810056


I am missing "Age" variable.

In [23]:
Y = trainframe[["Survived"]]

In [24]:
X = trainframe[["Pclass", "Sex", "Age", "SibSp", "Parch", "Embarked"]]

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 10)

In [26]:
model

LinearRegression(copy_X=True, fit_intercept=True, normalize=False)

In [27]:
model.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, normalize=False)

In [28]:
test_predictions = model.predict(X_test)

In [29]:
predictions = test_predictions
predictions[test_predictions < 0.5] = 0
predictions[test_predictions >=0.5] = 1

In [30]:
accurracy = sum(predictions[predictions == Y_test])/ len(predictions)

In [31]:
print(accurracy)

0.25139664804469275


In [84]:
Y = trainframe[["Survived"]]
X = trainframe[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.5, random_state = 10)

In [88]:
model.fit(X_train, Y_train)
test_predictions = model.predict(X_test)
len(X_test)

446

In [90]:
predictions = test_predictions
predictions[test_predictions < 0.2] = 0
predictions[test_predictions >=0.2] = 1
len(predictions)

446

In [91]:
accurracy = sum(predictions[predictions == Y_test])/ len(predictions)
print(accurracy)

0.2623318385650224


This seems unusually low. I have tried changing the threshold as well as predictors used. It seems I may have to be more explicit in how I set up the "Predictions" list as well as possibly flatten Y_test, since that is probably a numpy array. Repeating some steps.

In [142]:
Y = trainframe[["Survived"]]
X = trainframe[["Pclass", "Sex", "Age", "Embarked"]]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 10)

In [143]:
model.fit(X_train, Y_train)
test_predictions = model.predict(X_test)
predictions = [1 if x >0.5 else 0 for x in  test_predictions]
type(predictions)

list

In [144]:
Y_test_flat = np.ravel(Y_test).tolist()
type(Y_test_flat)

list

In [145]:
accurracy = [1 if predictions[i] == Y_test_flat[i] else 0 for i in range(len(predictions                                                                    ))]
print(sum(accurracy)/len(accurracy))

0.8268156424581006


This seems more reasonable as it is comparable to the result from  a kfold cross validation.

In [ ]:
We need the size of our data table so we can run the kfold cross validation

In [ ]:
framesize = trainframe.shape

In [44]:
framesize

(891, 12)

In [46]:
trainframe.size

10692

In [47]:
type(framesize)

tuple

In [48]:
framesize[0]

891

In [51]:
trainfolds = sklearn.cross_validation.KFold(framesize[0], n_folds = 3, random_state = 1)

In [52]:
predictions = []

In [53]:
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

In [54]:
model = LinearRegression()

In [55]:
for train, test in trainfolds:
    train_predictors = trainframe[predictors].iloc[train, :]
    train_survival = trainframe["Survived"].iloc[train]
    model.fit(train_predictors, train_survival)
    survival = model.predict(trainframe[predictors].iloc[test, :])
    predictions.append(survival)

In [57]:
predictions

[array([0.08998778098664151, 0.9607562062264055, 0.5926762776633075,
        0.9311387278910148, 0.05293430709559843, 0.17027568492773426,
        0.36994359030196217, 0.10347484742579838, 0.5215979058146017,
        0.8744910503206459, 0.6488836111621751, 0.8297427688347137,
        0.13479719839863458, -0.16112684364249996, 0.658141306629678,
        0.6398197484682683, 0.15173387493789658, 0.29543271790803693,
        0.5353779589276423, 0.6210076833082594, 0.2618725916383585,
        0.2626875613868225, 0.7317391597365727, 0.5059958971692387,
        0.5613985666552637, 0.3350397341633621, 0.13033880755757976,
        0.468765766528213, 0.660737752649524, 0.09108192184311426,
        0.4772239200354355, 1.042200261903615, 0.6606916127819146,
        0.08715392731116511, 0.5285507322778307, 0.4018743378420788,
        0.1303403074603967, 0.12933967231176546, 0.5727171285933563,
        0.6652388218334943, 0.4832157785469819, 0.7608074080287127,
        0.13057836346464147, 0.8718671

In [59]:
len(predictions)

3

In [60]:
type(predictions)

list

In [64]:
len(predictions[0])

297

Each of the folds returned the probabilty for a different test set, in a different column of the array. But they should be a one dimensional list.

In [65]:
predictions = np.concatenate(predictions, axis = 0 )

In [68]:
predictions[predictions <= 0.5] = 0
predictions[predictions > 0.5] = 1
accurracy = sum(predictions[predictions == trainframe["Survived"]])/ len(predictions)
print(accurracy)

0.7833894500561167


/home/avanti/anaconda/notebook/src/ipykernel/ipykernel/__main__.py:3: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  app.launch_new_instance()


NOt sure why there is such a big difference in the accuracy. 

In [69]:
trainfolds = sklearn.cross_validation.KFold(framesize[0], n_folds = 5, random_state = 1)
predictions = []
for train, test in trainfolds:
    train_predictors = trainframe[predictors].iloc[train, :]
    train_survival = trainframe["Survived"].iloc[train]
    model.fit(train_predictors, train_survival)
    survival = model.predict(trainframe[predictors].iloc[test, :])
    predictions.append(survival)
predictions = np.concatenate(predictions, axis = 0 )
predictions[predictions <= 0.5] = 0
predictions[predictions > 0.5] = 1
accurracy = sum(predictions[predictions == trainframe["Survived"]])/ len(predictions)
print(accurracy)

0.7878787878787878


/home/avanti/anaconda/notebook/src/ipykernel/ipykernel/__main__.py:12: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
